In [1]:
!pip install transformers datasets

     |████████████████████████████████| 3.4 MB 5.3 MB/s 
     |████████████████████████████████| 311 kB 42.0 MB/s 
     |████████████████████████████████| 3.3 MB 44.3 MB/s 
     |████████████████████████████████| 596 kB 47.0 MB/s 
     |████████████████████████████████| 67 kB 5.2 MB/s 
     |████████████████████████████████| 895 kB 41.2 MB/s 
     |████████████████████████████████| 243 kB 47.0 MB/s 
     |████████████████████████████████| 1.1 MB 40.3 MB/s 
     |████████████████████████████████| 133 kB 51.4 MB/s 
     |████████████████████████████████| 144 kB 46.4 MB/s 
     |████████████████████████████████| 94 kB 3.4 MB/s 
     |████████████████████████████████| 271 kB 48.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [3]:
from datasets import load_dataset, load_from_disk

In [54]:
dataset = load_dataset('csv', data_files="/gdrive/MyDrive/university/summaries_claims.csv")

Using custom data configuration default-2d4158dbfa783568
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-2d4158dbfa783568/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e)


  0%|          | 0/1 [00:00<?, ?it/s]

In [55]:
dataset = dataset.rename_column("summaries", "input_ids") \
  .rename_column("claims", "decoder_input_ids") \
  .remove_columns("patentnumber")
dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'decoder_input_ids'],
        num_rows: 29866
    })
})

In [56]:
dataset = dataset.filter(lambda r: r["input_ids"] is not None)

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-2d4158dbfa783568/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-7dde3fed1d8890f8.arrow


In [18]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("google/bigbird-pegasus-large-bigpatent")

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.35M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/775 [00:00<?, ?B/s]

In [57]:
def encoder_tokenize_function(row):
  return tokenizer(row["input_ids"], padding="max_length", truncation=True)

def decoder_tokenize_function(row):
  tokenized = tokenizer(row["decoder_input_ids"], padding="max_length", truncation=True)
  return {
      "decoder_input_ids": tokenized["input_ids"],
      "decoder_attention_mask": tokenized["attention_mask"]
  }


dataset = dataset.map(encoder_tokenize_function)
dataset = dataset.map(decoder_tokenize_function)

0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

In [79]:
dataset.save_to_disk("/gdrive/MyDrive/university/dataset")

In [4]:
dataset = load_from_disk("/gdrive/MyDrive/university/dataset")

In [5]:
from transformers import BigBirdPegasusForConditionalGeneration, AutoTokenizer
model = BigBirdPegasusForConditionalGeneration.from_pretrained("google/bigbird-pegasus-large-bigpatent", block_size=1, num_random_blocks=1)

Downloading:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.15G [00:00<?, ?B/s]

In [77]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

AttributeError: ignored

In [14]:
train = dataset["train"].select(list(range(100)))

In [16]:
!nvidia-smi

Thu Jan 27 17:15:24 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    56W / 149W |   2802MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [15]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments("test_trainer", num_train_epochs=1)
trainer = Trainer(
    model=model, 
    args=training_args, 
    train_dataset=train
)

In [17]:
trainer.train()

***** Running training *****
  Num examples = 100
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 13
/usr/local/lib/python3.7/dist-packages/transformers/models/bigbird_pegasus/modeling_bigbird_pegasus.py:792: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  * num_indices_to_pick_from


RuntimeError: ignored